## Wrangling Files and Data

Import pandas as pd:

In [2]:
import pandas as pd

### Working with Unstructured Data
From the Berkeley Earth surface temperature dataset, download localized data for New York City:

In [3]:
! curl -o ./data/nyc_temp.txt http://berkeleyearth.lbl.gov/auto/Local/TAVG/Text/40.99N-74.56W-TAVG-Trend.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  321k  100  321k    0     0   629k      0 --:--:-- --:--:-- --:--:--  629k


Inspect the file:

`! head ./data/nyc_temp.txt`

In [4]:
! head ./data/nyc_temp.txt

% This file contains an extracted local summary of land-surface 
% temperature results produced by the Berkeley Earth averaging 
% method for the location: 
% 
%    40.99 N, 74.56 W
% 
% The Berkeley Earth method takes temperature observations from a large 
% collection of weather monitoring stations and produces an estimate of 
% the underlying global temperature field across all of the Earth's 
% land areas.  Once this temperature field has been generated, it is 


In [10]:
temp = pd.read_csv('./data/nyc_temp.txt')

This file is not well-formatted. Use head/tail to narrow the selection and inspect the data:

`! head -72 nyc_temp.txt | tail -8`

In [6]:
! head -74 ./data/nyc_temp.txt | tail -8

%                  Monthly          Annual          Five-year        Ten-year        Twenty-year
% Year, Month,  Anomaly, Unc.,   Anomaly, Unc.,   Anomaly, Unc.,   Anomaly, Unc.,   Anomaly, Unc.
 
  1743     4    -1.847  1.848       NaN    NaN       NaN    NaN       NaN    NaN       NaN    NaN
  1743     5     1.064  1.348       NaN    NaN       NaN    NaN       NaN    NaN       NaN    NaN
  1743     6       NaN    NaN       NaN    NaN       NaN    NaN       NaN    NaN       NaN    NaN
  1743     7     3.137  1.282       NaN    NaN       NaN    NaN       NaN    NaN       NaN    NaN
  1743     8       NaN    NaN       NaN    NaN       NaN    NaN       NaN    NaN       NaN    NaN


### Reading and Formatting Files

Create a list of column names:

`col_names = ['year', 'month', 'monthly_anom']`

Supply the following arguments in `read_csv()`:

`header=None`

`usecols=[0,1,2]`

`names = col_names`

`delim_whitespace=True`

`comment="%"`

`temp = pd.read_csv('nyc_temp.txt', header=None, delim_whitespace=True, usecols=[0,1,2], names=col_names, comment="%")`



In [8]:
col_names = ['year', 'month', 'monthly_anom']
temp = pd.read_csv('./data/nyc_temp.txt', header=None, delim_whitespace=True, usecols=[0,1,2], names=col_names, comment="%")
temp

year  month  monthly_anom
0     1743      4        -1.847
1     1743      5         1.064
2     1743      6           NaN
3     1743      7         3.137
4     1743      8           NaN
...    ...    ...           ...
3322  2020      2         4.012
3323  2020      3         3.534
3324  2020      4        -1.183
3325  2020      5        -0.391
3326  2020      6         1.227

[3327 rows x 3 columns]

### Sorting

Sorting Data
Use sort_values to sort on a specific column using axis=0.

In [9]:
temp.sort_values("monthly_anom", axis=0)

year  month  monthly_anom
2290  1934      2        -7.116
1064  1831     12        -6.578
2960  1989     12        -6.174
1365  1857      1        -5.977
1703  1885      3        -5.907
...    ...    ...           ...
448   1780      8           NaN
449   1780      9           NaN
450   1780     10           NaN
451   1780     11           NaN
452   1780     12           NaN

[3327 rows x 3 columns]

### Create a DateTime Index

`date_df = temp.drop('monthly_anom', axis=1)`

`date_df["day"] = 1`

`date_index = pd.DatetimeIndex(pd.to_datetime(date_df))`

`temp = temp.set_index(date_index).drop(["year", "month"], axis=1)`

In [7]:
col_names = ['year', 'month', 'monthly_anom']
temp = pd.read_csv('./data/nyc_temp.txt', header=None, usecols=[0, 1, 2], names=col_names, delim_whitespace=True, comment='%')

date_df = temp.drop('monthly_anom', axis=1)
date_df["day"] = 1
date_index = pd.DatetimeIndex(pd.to_datetime(date_df))
temp = temp.set_index(date_index).drop(["year", "month"], axis=1)
temp

monthly_anom
1743-04-01        -1.847
1743-05-01         1.064
1743-06-01           NaN
1743-07-01         3.137
1743-08-01           NaN
...                  ...
2020-02-01         4.012
2020-03-01         3.534
2020-04-01        -1.183
2020-05-01        -0.391
2020-06-01         1.227

[3327 rows x 1 columns]

### Fill Missing Values

`temp = temp.fillna(method="ffill")`

In [8]:
temp = temp.fillna(method="ffill")
temp

monthly_anom
1743-04-01        -1.847
1743-05-01         1.064
1743-06-01         1.064
1743-07-01         3.137
1743-08-01         3.137
...                  ...
2020-02-01         4.012
2020-03-01         3.534
2020-04-01        -1.183
2020-05-01        -0.391
2020-06-01         1.227

[3327 rows x 1 columns]

## Working with Large Datasets

Using the `chunksize` argument of `read_csv` to parse or peform operations on large files

In [22]:
filename = "./data/311Cases_092020.csv"
c_size = 3000

for sf_chunk in pd.read_csv(filename, chunksize=c_size):
    print (sf_chunk.shape)

(3000, 47)
(3000, 47)
(3000, 47)
(3000, 47)
(3000, 47)
(3000, 47)
(3000, 47)
(3000, 47)
(3000, 47)
(3000, 47)
(3000, 47)
(1043, 47)


In [18]:
for sf_chunk in pd.read_csv(filename, chunksize=c_size):
    sf_chunk = sf_chunk.iloc[:, 0:18]
    sf_chunk = sf_chunk.fillna("None")
    date_index = pd.DatetimeIndex(pd.to_datetime(sf_chunk["Opened"]))
    sf_chunk.loc[:,"Opened"] = date_index
    sf_chunk.loc[:,"Weekday"] = date_index.day_name
    print (sf_chunk["Neighborhood"])

0        Inner Richmond
1       Mission Dolores
2       Sherwood Forest
3                Castro
4       Mission Dolores
             ...       
2995          Excelsior
2996            Bayview
2997          Sunnyside
2998            Bayview
2999            Portola
Name: Neighborhood, Length: 3000, dtype: object
3000           Chinatown
3001             Bayview
3002             Bayview
3003    Western Addition
3004             Mission
              ...       
5995     South of Market
5996             Mission
5997           Panhandle
5998             Mission
5999                None
Name: Neighborhood, Length: 3000, dtype: object
6000      Golden Gate Park
6001    Financial District
6002               Mission
6003          Potrero Hill
6004       South of Market
               ...        
8995               Portola
8996          Union Street
8997        Outer Richmond
8998        Crocker Amazon
8999        Outer Richmond
Name: Neighborhood, Length: 3000, dtype: object
9000      Outer Rich

### NaN Functions

`import numpy as np`

`df = pd.DataFrame(np.random.randn(3, 3), index=['a', 'c', 'e'], columns=['one','two', 'three'])`

`print ("Original" + "\n")`

`print (df)`

`df = df.reindex(['a', 'b', 'c'])`

`print ("\n"  + "Reindexed:" + '\n')`

`print (df)`

`print ("\n"  + "NaN replaced with '0':" + "\n")`

`print (df.fillna(0))`

In [20]:
import numpy as np

df = pd.DataFrame(np.random.randn(3, 3), index=['a', 'c', 'e'], columns=['one','two', 'three'])
print ("Original" + "\n")
print (df)

df = df.reindex(['a', 'b', 'c'])
print ("\n"  + "Reindexed:" + '\n')
print (df)

print ("\n"  + "NaN replaced with '0':" + "\n")
print (df.fillna(0))

Original

        one       two     three
a -0.895244  0.413684  0.559613
c  0.758809  2.197271 -0.581800
e -0.068908  0.603417 -0.271750

Reindexed:

        one       two     three
a -0.895244  0.413684  0.559613
b       NaN       NaN       NaN
c  0.758809  2.197271 -0.581800

NaN replaced with '0':

        one       two     three
a -0.895244  0.413684  0.559613
b  0.000000  0.000000  0.000000
c  0.758809  2.197271 -0.581800


In [ ]:
df = pd.DataFrame(np.random.randn(3, 3), index=['a', 'c', 'e'], columns=['one','two', 'three'])
print ("Original" + "\n")
print (df)
df = df.reindex(['a', 'b', 'c'])
print ("\n"  + "Reindexed:" + '\n')
print (df)
df = df.dropna()
print ("\n"  + "After Dropping NaN rows:" + '\n')
print (df)